<a href="https://colab.research.google.com/github/Eng-abdelhamed/Exercises-and-Assignmentss/blob/main/4__exercise_grouping_and_sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In these exercises we'll apply groupwise analysis to our dataset.

Run the code cell below to load the data before running the exercises.

In [5]:
import pandas as pd

reviews = pd.read_csv("winemag-data_first150k.csv", index_col=0,chunksize = 10000)
reviews = next(reviews)
reviews.head()
#pd.set_option("display.max_rows", 5)

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


# Exercises

## 1.
Who are the most common wine reviewers in the dataset? Create a `Series` whose index is the `taster_twitter_handle` category from the dataset, and whose values count how many reviews each person wrote.

In [6]:
# Your code here
reviews_written = reviews.groupby('designation').apply(lambda df: df.designation.count())
reviews_written


<ipython-input-6-f3b6afee64e5>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reviews_written = reviews.groupby('designation').apply(lambda df: df.designation.count())


,0
designation,
'S',2
075 Carati,1
10 Anos,1
10 Anos Tawny,1
100% Skin Fermented,1
...,...
Òlbia,1
Único,1
Ürziger Goldwingert Spätlese,1


## 2.
What is the best wine I can buy for a given amount of money? Create a `Series` whose index is wine prices and whose values is the maximum number of points a wine costing that much was given in a review. Sort the values by price, ascending (so that `4.0` dollars is at the top and `3300.0` dollars is at the bottom).

In [8]:
best_rating_per_price = reviews.groupby('price')['points'].max().sort_index(ascending = True)
best_rating_per_price

,points
price,
4.0,83
5.0,81
6.0,87
7.0,89
8.0,88
...,...
520.0,95
698.0,97
740.0,93


## 3.
What are the minimum and maximum prices for each `variety` of wine? Create a `DataFrame` whose index is the `variety` category from the dataset and whose values are the `min` and `max` values thereof.

In [9]:
price_extremes = reviews.groupby('variety').agg({'price':['max','min']})
price_extremes


price      
                     max   min
variety                       
Agiorgitiko         25.0  12.0
Aglianico           90.0  14.0
Albariño            40.0  15.0
Alfrocheiro         32.0  14.0
Alicante Bouschet  130.0   7.0
...                  ...   ...
Yapincak            16.0  16.0
Zibibbo             20.0  19.0
Zinfandel          100.0  10.0
Zweigelt            32.0  10.0
Žilavka             15.0  13.0

[328 rows x 2 columns]

## 4.
What are the most expensive wine varieties? Create a variable `sorted_varieties` containing a copy of the dataframe from the previous question where varieties are sorted in descending order based on minimum price, then on maximum price (to break ties).

In [20]:
sorted_varieties = price_extremes.sort_values(by=[('price', 'min'), ('price', 'max')], ascending=[False, False])
sorted_varieties

price       
                                  max    min
variety                                     
Cabernet-Shiraz                 150.0  150.0
Shiraz-Viognier                 125.0  125.0
Merlot-Cabernet Sauvignon        96.0   96.0
Sangiovese-Syrah                 66.0   66.0
Carignan-Grenache                65.0   65.0
...                               ...    ...
Roviello                          NaN    NaN
Sacy                              NaN    NaN
Sauvignon Blanc-Sauvignon Gris    NaN    NaN
Touriga Nacional Blend            NaN    NaN
Vinhão                            NaN    NaN

[328 rows x 2 columns]

## 5.
Create a `Series` whose index is reviewers and whose values is the average review score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [26]:
reviewer_mean_ratings = reviews.groupby('winery').points.mean()
reviewer_mean_ratings


,points
winery,
1000 Stories,90.000000
10Span,83.000000
12 Linajes,90.000000
14 Hands,87.833333
1848 Winery,89.000000
...,...
the Ghost of 413,86.000000
Çamlibağ,86.500000
Écluse,91.000000


Are there significant differences in the average scores assigned by the various reviewers? Run the cell below to use the `describe()` method to see a summary of the range of values.

In [27]:
reviewer_mean_ratings.describe()

,points
count,4305.000000
mean,88.384852
std,2.548228
min,80.000000
25%,87.000000
50%,88.000000
75%,90.000000
max,98.000000


## 6.
What combination of countries and varieties are most common? Create a `Series` whose index is a `MultiIndex`of `{country, variety}` pairs. For example, a pinot noir produced in the US should map to `{"US", "Pinot Noir"}`. Sort the values in the `Series` in descending order based on wine count.

In [35]:
country_variety_counts = reviews.groupby(['country','variety']).size().sort_values(ascending = False)
country_variety_counts


country  variety                 
US       Pinot Noir                  900
         Chardonnay                  552
         Cabernet Sauvignon          472
France   Bordeaux-style Red Blend    442
Italy    Nebbiolo                    314
                                    ... 
Spain    Jaen                          1
         Listán Negro                  1
Greece   Malagouzia                    1
Spain    Malvasia                      1
Lebanon  Rosé                          1
Length: 605, dtype: int64

# Keep going
